# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [2]:
import pandas as pd
import numpy as np
import re
import os
import random
import transformers as ppb
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import f1_score as f1, confusion_matrix, recall_score as recall, precision_score as precision
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, cv, Pool
from tqdm import notebook

In [3]:
def set_seed(seed=1):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(1)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Подготовка

Загрузим датасет с отзывами:

In [4]:
try:
    dataset = pd.read_csv('datasets/toxic_comments.csv').copy()
except:
    dataset = pd.read_csv('/datasets/toxic_comments.csv').copy()

In [5]:
dataset.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [14]:
dataset.query('toxic == 1')['text'].head(50)

6           COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
12     Hey... what is it..\n@ | talk .\nWhat is it......
16     Bye! \n\nDon't look, come or think of comming ...
42     You are gay or antisemmitian? \n\nArchangel WH...
43              FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
44     I'm Sorry \n\nI'm sorry I screwed around with ...
51     GET FUCKED UP. GET FUCKEEED UP.  GOT A DRINK T...
55     Stupid peace of shit stop deleting my stuff as...
56     =Tony Sidaway is obviously a fistfuckee. He lo...
58     My Band Page's deletion. You thought I was gon...
59     Why can't you believe how fat Artie is? Did yo...
65     All of my edits are good.  Cunts like you who ...
79     Hi! I am back again!\nLast warning!\nStop undo...
86     Would you both shut up, you don't run wikipedi...
105           A pair of jew-hating weiner nazi schmucks.
151    "\n\nSORRY PUCK BUT NO ONE EVER SAID DICK WAS ...
159    "\n\nUNBLOCK ME OR I'LL GET MY LAWYERS ON TO Y...
168    You should be fired, you

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [16]:
dataset = dataset[['text', 'toxic']]

### Балансировка классов

Предварительно было опробовано несколько моделей. Выяснилось, что в датасете присутствует проблема с дисбалансом классов.

In [17]:
dataset['toxic'].mean()

0.10161213369158527

Негативные комментарии составляют 10% всего датасета, учтем это в дальнейшем и реализуем увеличение обучающей выборки, когда получим эмбеддинги от BERT.

### Очистка комментариев

Напишем функцию для очистки текста от лишних символов, используя регулярные выражения. Далее будем использовать ее через `.apply()`

In [18]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    return " ".join(text.split())

Проверим ее работоспособность:

In [19]:
dataset['text'][:5]

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object

In [20]:
dataset['text'][:5].apply(clear_text)

0    Explanation Why the edits made under my userna...
1    D aww He matches this background colour I m se...
2    Hey man I m really not trying to edit war It s...
3    More I can t make any real suggestions on impr...
4    You sir are my hero Any chance you remember wh...
Name: text, dtype: object

**Остаются только символы латиницы, функция работает корректно.**

### Создание эмбеддингов

Импортируем предобученную модель `ToxicBERT` и токенизатор, с помощью них будем создавать признаки для наших стандартных моделей классификации.

In [11]:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

##для BERT вместо distilBERT
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

#Загрузка модели и токенизатора
#tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

**Теперь начнём преобразование текстов в эмбеддинги.** <br>
Функция ниже генерирует подвыборку из изначально обработанного датасета, производит токенизацию, а также возвращает таргеты для данной подвыборки.

In [13]:
def preparation(dataset, sample_size):
    data = dataset.sample(sample_size).reset_index(drop=True).copy()
    data['text'] = data['text'].apply(clear_text)
    
    tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=128, 
                                                               padding=True, truncation=True)))
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
            
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    return padded, data['toxic']

Далее обработанный датасет передается в BERT, эмбеддинги поочередно генерируются и сохраняются.

In [14]:
%%time
set_seed(41)

sample_size = 5000
embeddings = []

padded, target = preparation(dataset, sample_size)
dataloader = DataLoader(torch.tensor(padded), batch_size=20)
#attention_mask = np.where(padded != 0, 1, 0)
    
for batch in notebook.tqdm(dataloader):
    attention_mask_batch = torch.LongTensor(np.where(batch != 0, 1, 0))
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
       
    embeddings.append(batch_embeddings[0].numpy())
    #embeddings.append(batch_embeddings[0][:,0,:].numpy())
    del batch
    del attention_mask_batch
    del batch_embeddings

  0%|          | 0/250 [00:00<?, ?it/s]

CPU times: user 10min 30s, sys: 1min 5s, total: 11min 35s
Wall time: 4min


In [15]:
features = np.concatenate(embeddings)

In [16]:
features.shape

(5000, 6)

In [17]:
features

array([[-6.6577835 , -9.195709  , -8.637882  , -8.82751   , -8.501408  ,
        -8.801255  ],
       [ 1.6300064 , -5.4017816 , -2.4157476 , -3.535969  , -2.6170852 ,
        -5.297613  ],
       [-7.0951786 , -9.149456  , -8.664189  , -9.103351  , -8.610858  ,
        -8.887666  ],
       ...,
       [-7.07103   , -9.120788  , -8.5948305 , -9.088688  , -8.618527  ,
        -8.858386  ],
       [-7.277208  , -9.042899  , -8.625817  , -8.869639  , -8.613976  ,
        -8.8847885 ],
       [-0.84738773, -8.1565    , -5.553962  , -6.65841   , -4.136997  ,
        -7.0153437 ]], dtype=float32)

In [18]:
target = np.array(target).astype('int')

In [19]:
target.shape

(5000,)

**Соотношение длин массивов соблюдено, теперь можно разбить выборки на трейн, тест и сделать upsample обучающей выборки.**

In [20]:
def upsample(features, target, repeat=1):
    features, target = pd.DataFrame(features), pd.Series(target)
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=1)
    
    return features_upsampled, target_upsampled

Также воспользуемся стратификацией, чтобы в тестовой выборке гарантированно сохранилось соотношение классов, представленное в исходном датасете.

In [21]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=1, stratify=target)

In [22]:
x_train, y_train = upsample(x_train, y_train, repeat=8)

In [23]:
y_train.mean()

0.4721663313212609

In [24]:
y_test.mean()

0.10066666666666667

**Теперь в обучающей выборке примерно равное соотношение классов, а в тестовой реально наблюдающееся по датасету.**

## Обучение. Кросс-валидация

Обучим модели логистической регрессии, случайного леса и градиентный бустинг CatBoost. Балансировку классов в них проводить уже не требуется.

In [25]:
def scores(y_true, y_pred):
    print('f1 =', format(f1(y_true, y_pred), '.2f'))
    print('Precision =', format(precision(y_true, y_pred), '.3f'))
    print('Recall =', format(recall(y_true, y_pred), '.3f'))
    print(confusion_matrix(y_true, y_pred).ravel(), '\n  tn  fp  fn  tp') #tn fp fn tp

### Логистическая регрессия

In [26]:
%%time
lg = LogisticRegression()

lg_grid = GridSearchCV(lg, param_grid={'max_iter': [500]}, cv=3, verbose=3, scoring='f1')

lg_grid.fit(x_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ......................max_iter=500;, score=0.978 total time=   0.1s
[CV 2/3] END ......................max_iter=500;, score=0.980 total time=   0.0s
[CV 3/3] END ......................max_iter=500;, score=0.979 total time=   0.0s
CPU times: user 676 ms, sys: 8.11 ms, total: 684 ms
Wall time: 107 ms


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'max_iter': [500]}, scoring='f1', verbose=3)

In [27]:
print(f'F1_score = {(lg_grid.best_score_):.3f}')

F1_score = 0.979


### Случайный лес

In [28]:
%%time
set_seed(1)

rf = RandomForestClassifier()

param_grid = {'n_estimators': [100, 150, 200], 
              'max_depth': [i for i in range(2,9,2)]}

rf_grid = GridSearchCV(rf, param_grid=param_grid, cv=3, scoring='f1', verbose=0)

rf_grid.fit(x_train, y_train)

CPU times: user 9.26 s, sys: 469 ms, total: 9.72 s
Wall time: 8.64 s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'n_estimators': [100, 150, 200]},
             scoring='f1')

In [29]:
print(f'F1_score = {rf_grid.best_score_:.3f}')
print(rf_grid.best_params_)

F1_score = 0.990
{'max_depth': 8, 'n_estimators': 200}


### Градиентный бустинг CatBoost

In [30]:
cat_train = Pool(x_train, y_train)
cat_model = cat = CatBoostClassifier(verbose=0)

cat_params = {'learning_rate': [0.1, 0.2, 0.3],
              'n_estimators': [100, 200],
              'depth': [2, 3]
}

In [31]:
%%time
set_seed(1)

cat_grid_search = cat_model.grid_search(cat_params, cat_train, cv=3, partition_random_seed=0, plot=False)


bestTest = 0.04668815865
bestIteration = 99

0:	loss: 0.0466882	best: 0.0466882 (0)	total: 198ms	remaining: 2.18s

bestTest = 0.03722060363
bestIteration = 99

1:	loss: 0.0372206	best: 0.0372206 (1)	total: 322ms	remaining: 1.61s

bestTest = 0.03506341939
bestIteration = 92

2:	loss: 0.0350634	best: 0.0350634 (2)	total: 465ms	remaining: 1.4s

bestTest = 0.0391660981
bestIteration = 199

3:	loss: 0.0391661	best: 0.0350634 (2)	total: 724ms	remaining: 1.45s

bestTest = 0.03343062253
bestIteration = 192

4:	loss: 0.0334306	best: 0.0334306 (4)	total: 972ms	remaining: 1.36s

bestTest = 0.03286884881
bestIteration = 184

5:	loss: 0.0328688	best: 0.0328688 (5)	total: 1.24s	remaining: 1.24s

bestTest = 0.03598275881
bestIteration = 99

6:	loss: 0.0359828	best: 0.0328688 (5)	total: 1.36s	remaining: 970ms

bestTest = 0.03047267087
bestIteration = 96

7:	loss: 0.0304727	best: 0.0304727 (7)	total: 1.52s	remaining: 760ms

bestTest = 0.03025517205
bestIteration = 96

8:	loss: 0.0302552	best: 0.030255

In [32]:
cat_grid_search['params']

{'depth': 3, 'iterations': 200, 'learning_rate': 0.2}

In [43]:
cv_data = cv(
    params={'depth': 3, 'iterations': 200, 'learning_rate': 0.2, 'loss_function': 'Logloss', 'eval_metric' : 'F1'},
    pool=cat_train,
    fold_count=3,
    partition_random_seed=0,
    plot=False,
    verbose=False
)

Training on fold [0/3]

bestTest = 0.9889415482
bestIteration = 68

Training on fold [1/3]

bestTest = 0.9910290237
bestIteration = 111

Training on fold [2/3]

bestTest = 0.9936440678
bestIteration = 108



**Финальный скор:**

In [50]:
print('F1_score =', round(cv_data['test-F1-mean'].mean(), 4))

F1_score = 0.9883


In [48]:
%%time

cat = CatBoostClassifier(iterations=200, depth=2, learning_rate=0.2, verbose=50)

cat.fit(x_train, y_train)
cat_pred = cat.predict(x_test)

0:	learn: 0.3812395	total: 3.33ms	remaining: 664ms
50:	learn: 0.0388910	total: 81.7ms	remaining: 239ms
100:	learn: 0.0272673	total: 155ms	remaining: 152ms
150:	learn: 0.0215978	total: 228ms	remaining: 74.1ms
199:	learn: 0.0180508	total: 300ms	remaining: 0us
CPU times: user 437 ms, sys: 406 ms, total: 842 ms
Wall time: 316 ms


In [49]:
print(f'F1_score = {(lg_grid.best_score_):.3f} - Логистическая регрессия')
print(f'F1_score = {rf_grid.best_score_:.3f} - Случайный лес')
print('F1_score =', round(cv_data['test-F1-mean'].mean(), 3), '- CatBoost')

F1_score = 0.979 - Логистическая регрессия
F1_score = 0.990 - Случайный лес
F1_score = 0.988 - CatBoost


**Все модели удовлетворяют требованию к результату f1-меры выше 0.75, и у всех примерно одинаковый результат на кросс-валидации. Для окончательного тестирования выберем модель случайного леса — она обладает наилучшим качеством и при этом достаточно быстрая.**

### Тестирование модели

Модель случайного леса со следующими параметрами:

In [51]:
print(f'F1_score = {rf_grid.best_score_:.3f}')
print(rf_grid.best_params_)

F1_score = 0.990
{'max_depth': 8, 'n_estimators': 200}


**На тестовой выборке:**

In [54]:
y_test.sum() #всего негативных отзывов

151

In [53]:
scores(y_test, rf_grid.best_estimator_.predict(x_test))

f1 = 0.92
Precision = 0.903
Recall = 0.927
[1334   15   11  140] 
  tn  fp  fn  tp


Качество метрики `f1 = 0.92`, модель отлично справляется с отловом токсичных комментариев.

## Вывод

Несмотря на обилие данных, нейронной сети `ToxicBERT` хватает даже 1000 текстов, чтобы дальнейшие модели на полученных эмбеддингах хорошо выявляли негативные сообщения. Главная проблема заключалась в устранении дисбаланса классов, — если модель не будет видеть достаточно примеров какого-либо класса, будет сложно обучить ее на высокое значение метрики. Наилучшей оказалась модель случайного леса глубиной 8 и количеством деревьев 200 — `f1_test = 0.92`. Модель хорошо выявляет как положительные, так и отрицательные классы. Из 151 негативного комментария не было выявлено всего 11 (FN=11), это составляет ~7%.